# Final Model Prediction

In [44]:
%cd '/UltimeTradingBot/Crypto_backtest_tools'
# Normalization_File='/UltimeTradingBot/Data/w20_004_Noramalization_val.json'
# Model_FileName='/UltimeTradingBot/Data/W20_004_Model.hdf5'


Normalization_File='w15_NoVol_Normalization.json'
Model_FileName='w15_NoVol_XcryptoAi_model.hdf5'
WINDOW_SIZE=15
window=WINDOW_SIZE
MAX_FORCAST_SIZE=7
BUY_PERCENT=0.6
SELL_PERCENT=2
NORM_FILE=Normalization_File
MODEL_FILE=Model_FileName
ALLHIST_FILE='Results_history.json'
DATA_DIR='/UltimeTradingBot/Data/'



/UltimeTradingBot/Crypto_backtest_tools


In [23]:
import sys
sys.path.append('/UltimeTradingBot/Crypto_backtest_tools')
%cd '/UltimeTradingBot/Crypto_backtest_tools'
import pandas as pd
import json
import time
import timeit
import numpy as np
import urllib
#import ccxt
import  ccxt  
import random
from keras.models import load_model
import os
import asyncio
import tensorflow as tf

/UltimeTradingBot/Crypto_backtest_tools


## Garbage collection

In [24]:
import gc
gc.collect()    

import warnings
warnings.filterwarnings('ignore')
PRERR=False
def prerr(err):
    if PRERR:
        print("\033[0;31m Error in "+str(sys._getframe().f_code.co_name) +" \033[0;33m"+str(err))

PDEBUG=True
def pdebug(err):
    if PDEBUG:
        print("\033[0;31m Error in "+str(sys._getframe().f_code.co_name) +" \033[0;33m"+str(err))

## Gloabale variables

In [25]:
Normalization=None
Model_FileName=Model_FileName
Normalization_File=Normalization_File
Binance_USDT_HALAL = [
    "LUNA/USDT",
    "ETH/USDT",
    "GMT/USDT",
    "UST/USDT",
    "SOL/USDT",
    "APE/USDT",
    "XRP/USDT",
    "IDEX/USDT",
    "AVAX/USDT",
    "DOT/USDT",
    "ADA/USDT",
    "JASMY/USDT",
    "TRX/USDT",
    "NEAR/USDT",
    "AXS/USDT",
    "GAL/USDT",
    "GALA/USDT",
    "SHIB/USDT",
    "ZIL/USDT",
    "ENS/USDT",
    "DOGE/USDT",
    "LTC/USDT",
    "EUR/USDT",
    "MANA/USDT",
    "DAR/USDT",
    "WAVES/USDT",
    "LAZIO/USDT",
    "ALICE/USDT",
    "ROSE/USDT",
    "ZEC/USDT",
    "ALGO/USDT",
    "GRT/USDT",
    "PSG/USDT",
    "SLP/USDT",
    "EOS/USDT",
    "PORTO/USDT",
    "ICP/USDT",
    "EGLD/USDT",
    "XMR/USDT",
    "KDA/USDT",
    "ETC/USDT",
    "MBOX/USDT",
    "OGN/USDT",
    "AR/USDT",
    "GLMR/USDT",
    "LOKA/USDT",
    "XLM/USDT",
    "MTL/USDT",
    "SNX/USDT",
    "PYR/USDT",
    "DASH/USDT",
    "CITY/USDT",
    "ASTR/USDT",
    "IOTA/USDT",
    "ACM/USDT",
    "BAR/USDT",
    "JUV/USDT",
    "SYS/USDT",
    "RVN/USDT",
    "MBL/USDT",
    "REN/USDT",
    "JST/USDT",
    "OMG/USDT",
    "ATM/USDT",
    "XEC/USDT",
    "STORJ/USDT",
    "ZRX/USDT",
    "SRM/USDT",
    "ICX/USDT",
    "API3/USDT",
    "ONT/USDT",
    "SKL/USDT",
    "MULTI/USDT",
    "QTUM/USDT",
    "COCOS/USDT",
    "VOXEL/USDT",
    "HIVE/USDT",
    "KP3R/USDT",
    "ATA/USDT",
    "STMX/USDT",
    "ADX/USDT",
    "HIGH/USDT",
    "NULS/USDT",
    "MLN/USDT",
    "YGG/USDT",
    "SC/USDT",
    "CKB/USDT",
    "TOMO/USDT",
    "STX/USDT",
    "FLUX/USDT",
    "DNT/USDT",
    "ORN/USDT",
    "PLA/USDT",
    "BADGER/USDT",
    "DF/USDT",
    "MOB/USDT",
    "LPT/USDT",
    "SCRT/USDT",
    "RAD/USDT",
    "NMR/USDT",
    "ELF/USDT",
    "TORN/USDT",
    "T/USDT",
    "QUICK/USDT",
    "LSK/USDT",
    "FIDA/USDT",
    "XNO/USDT",
    "BTG/USDT",
    "GHST/USDT",
    "EPS/USDT"
]

pair_list = Binance_USDT_HALAL


## Local Functions first

In [26]:
# Get list of all IDs on binance
def give_first_kline_open_stamp(interval, symbol, start_ts=1499990400000):
        '''
        Returns the first kline from an interval and start timestamp and symbol
        :param interval:  1w, 1d, 1m etc - the bar length to query
        :param symbol:    BTCUSDT or LTCBTC etc
        :param start_ts:  Timestamp in miliseconds to start the query from
        :return:          The first open candle timestamp
        '''

        url_stub = "http://api.binance.com/api/v1/klines?interval="

        #/api/v1/klines?interval=1m&startTime=1536349500000&symbol=ETCBNB
        addInterval   = url_stub     + str(interval) + "&"
        addStarttime  = addInterval   + "startTime="  + str(start_ts) + "&"
        addSymbol     = addStarttime + "symbol="     + str(symbol)
        url_to_get = addSymbol

        kline_data = urllib.request.urlopen(url_to_get).read().decode("utf-8")
        kline_data = json.loads(kline_data)

        return kline_data[0][0]

def get_crypto_metadata(pair_list):
    Binance_USDT_HALAL=pair_list
    ids = []
    #ids = all_ids()
    for halalpair in Binance_USDT_HALAL:
    #    print( halalpair.replace('/',''))
        ids.append(halalpair.replace('/',''))
    #print(ids)
    MetaData=pd.DataFrame(ids)
    MetaData["Pair"]=Binance_USDT_HALAL
    counters=0
    for this_id in ids:
        '''
        Find launch Week of symbol, start at Binance launch date 2017-07-14 (1499990400000)
        Find launch Day of symbol in week
        Find launch minute of symbol in day
        '''

        symbol_launch_week_stamp   = give_first_kline_open_stamp('1w', this_id, 1499990400000 )
        symbol_launch_day_stamp    = give_first_kline_open_stamp('1d', this_id, symbol_launch_week_stamp)
        symbol_launch_minute_stamp = give_first_kline_open_stamp('1m', this_id, symbol_launch_day_stamp)
        MetaData.loc[counters,"launch_week_stamp"]=str(symbol_launch_week_stamp)
        MetaData.loc[counters,"launch_day_stamp"]=str(symbol_launch_day_stamp)
        MetaData.loc[counters,"launch_minute"]=pd.to_datetime(symbol_launch_minute_stamp, unit='ms')

        counters += 1

        #print("Week stamp", symbol_launch_week_stamp)
        #print("Day  stamp", symbol_launch_day_stamp)
        #print("Min  stamp", symbol_launch_minute_stamp)
        print(counters,end=" ")
        #print(this_id, "launched", symbol_launch_minute_stamp )
    return MetaData
    #print("")
    
def normalize(dataset,file=Normalization_File):
    global Normalization
    try:
        N=Normalization
    except:
        Normalization=None
    if(Normalization==None):
        #print('Loading normalization from file')
        with open(file) as json_file:
            Normalization = json.load(json_file)
    else:
        #print('normalization is loaded')
        pass

    mean=np.array(Normalization["mean"])
    std=np.array(Normalization["std"])
    dataset -= mean 
    dataset /= std
    return(dataset)

def Buy_Dessision(input):
    A=np.array(input)
    A = A.reshape(1,A.shape[0])
    predictions = model.predict(normalize(A))
    rounded = [round(x[0]) for x in predictions]
    return(rounded[0])

def Buy_Dessision_Normalized(input):
    A=np.array(input)
    A = A.reshape(1,A.shape[0])
    predictions = model.predict(A)
    rounded = [round(x[0]) for x in predictions]
    return(rounded[0])

def Buy_Dessision_Multi_In_Out(input):
    A=np.array(input)
    predictions = model.predict(normalize(A))
    rounded = [round(x[0]) for x in predictions]
    return(rounded)

def Buy_Dessision_Multi_In_Out_Normalized(input):
    A=np.array(input)
    predictions = model.predict(A)
    rounded = [round(x[0]) for x in predictions]
    return(rounded)

def instant_pair_data(pair="GMT/BUSD",exchange=ccxt.binance(),window=WINDOW_SIZE):
    ex=exchange
    ticker = ex.fetch_ticker(pair)
    pair_current_price=ticker['info']['askPrice']
    #print(pair_current_price)

    ohlcv1m = ex.fetch_ohlcv(pair, '1m', limit=window)
    ohlcv5m = ex.fetch_ohlcv(pair, '5m', limit=window)
    ohlcv15m = ex.fetch_ohlcv(pair, '15m', limit=window)
    ohlcv1h = ex.fetch_ohlcv(pair, '1h', limit=window)
    ohlcv1d = ex.fetch_ohlcv(pair, '1d', limit=window)

    pair_data=pd.DataFrame()
    pair_data.loc[0,"price"]=float(pair_current_price)
    #minute
    for window_i in range(1,window+1):
        pair_data.loc[0,"high-"+str(window_i)]=ohlcv1m[-window_i][1]
        pair_data.loc[0,"low-"+str(window_i)]=ohlcv1m[-window_i][2]
        pair_data.loc[0,"open-"+str(window_i)]=ohlcv1m[-window_i][0]
        pair_data.loc[0,"close-"+str(window_i)]=ohlcv1m[-window_i][3]
        if(window_i!=1):pair_data.loc[0,"volume-"+str(window_i)]=ohlcv1m[-window_i][4]

    for window_i in range(1,window+1):
        pair_data.loc[0,"high-"+str(window_i)+"_day"]=ohlcv1d[-window_i][1]
        pair_data.loc[0,"low-"+str(window_i)+"_day"]=ohlcv1d[-window_i][2]
        pair_data.loc[0,"open-"+str(window_i)+"_day"]=ohlcv1d[-window_i][0]
        pair_data.loc[0,"close-"+str(window_i)+"_day"]=ohlcv1d[-window_i][3]
        if(window_i!=1):pair_data.loc[0,"volume-"+str(window_i)+"_day"]=ohlcv1d[-window_i][4]  

    for window_i in range(1,window+1):
        pair_data.loc[0,"high-"+str(window_i)+"_hour"]=ohlcv1h[-window_i][1]
        pair_data.loc[0,"low-"+str(window_i)+"_hour"]=ohlcv1h[-window_i][2]
        pair_data.loc[0,"open-"+str(window_i)+"_hour"]=ohlcv1h[-window_i][0]
        pair_data.loc[0,"close-"+str(window_i)+"_hour"]=ohlcv1h[-window_i][3]
        if(window_i!=1):pair_data.loc[0,"volume-"+str(window_i)+"_hour"]=ohlcv1h[-window_i][4]  

    for window_i in range(1,window+1):
        pair_data.loc[0,"high-"+str(window_i)+"_15min"]=ohlcv15m[-window_i][1]
        pair_data.loc[0,"low-"+str(window_i)+"_15min"]=ohlcv15m[-window_i][2]
        pair_data.loc[0,"open-"+str(window_i)+"_15min"]=ohlcv15m[-window_i][0]
        pair_data.loc[0,"close-"+str(window_i)+"_15min"]=ohlcv15m[-window_i][3]
        if(window_i!=1):pair_data.loc[0,"volume-"+str(window_i)+"_15min"]=ohlcv5m[-window_i][4]  

    for window_i in range(1,window+1):
        pair_data.loc[0,"high-"+str(window_i)+"_5min"]=ohlcv5m[-window_i][1]
        pair_data.loc[0,"low-"+str(window_i)+"_5min"]=ohlcv5m[-window_i][2]
        pair_data.loc[0,"open-"+str(window_i)+"_5min"]=ohlcv5m[-window_i][0]
        pair_data.loc[0,"close-"+str(window_i)+"_5min"]=ohlcv5m[-window_i][3]
        if(window_i!=1):pair_data.loc[0,"volume-"+str(window_i)+"_5min"]=ohlcv5m[-window_i][4]   
    return  pair_data


# Getting Vars

In [27]:
try:
    MetaData=pd.read_csv("../Data/MetaData.csv")
except:
    MetaData=get_crypto_metadata(Binance_USDT_HALAL)


## loding the model

In [51]:
model = load_model(Model_FileName)
print(Model_FileName+' Loaded')

/UltimeTradingBot/Data/W20_004_Model.hdf5 Loaded


In [58]:
with open(Normalization_File) as json_file:
            Normalization = json.load(json_file)
print(len(Normalization['mean']))

996


In [29]:
Exchange=ccxt.binance()
ex=Exchange
exchange=Exchange


In [30]:
# bitcoin_ticker = Exchange.fetch_ticker('BTC/USDT')
# btc_current_price=bitcoin_ticker['info']['askPrice']
# print(btc_current_price)




# li1 = df.keys().values.tolist()
# li2 = fulldata.keys().values.tolist()
 
# s = set(li2)
# temp3 = [x for x in li1 if x not in s]
# print(temp3)

## Working Fonctions Deffinition

In [31]:

def instant_pair_data(pair="GMT/BUSD",exchange=ccxt.binance(),window=WINDOW_SIZE):
    ex=exchange
    ticker = ex.fetch_ticker(pair)
    pair_current_price=ticker['info']['askPrice']
    #print(pair_current_price)

    ohlcv1m = ex.fetch_ohlcv(pair, '1m', limit=window)
    ohlcv5m = ex.fetch_ohlcv(pair, '5m', limit=window)
    ohlcv15m = ex.fetch_ohlcv(pair, '15m', limit=window)
    ohlcv1h = ex.fetch_ohlcv(pair, '1h', limit=window)
    ohlcv1d = ex.fetch_ohlcv(pair, '1d', limit=window)

    pair_data=pd.DataFrame()
    pair_data.loc[0,"price"]=float(pair_current_price)
    #minute
    for window_i in range(1,window+1):
        pair_data.loc[0,"high-"+str(window_i)]=ohlcv1m[-window_i][1]
        pair_data.loc[0,"low-"+str(window_i)]=ohlcv1m[-window_i][2]
        pair_data.loc[0,"open-"+str(window_i)]=ohlcv1m[-window_i][0]
        pair_data.loc[0,"close-"+str(window_i)]=ohlcv1m[-window_i][3]
        if(window_i!=1):pair_data.loc[0,"volume-"+str(window_i)]=ohlcv1m[-window_i][4]

    for window_i in range(1,window+1):
        pair_data.loc[0,"high-"+str(window_i)+"_day"]=ohlcv1d[-window_i][1]
        pair_data.loc[0,"low-"+str(window_i)+"_day"]=ohlcv1d[-window_i][2]
        pair_data.loc[0,"open-"+str(window_i)+"_day"]=ohlcv1d[-window_i][0]
        pair_data.loc[0,"close-"+str(window_i)+"_day"]=ohlcv1d[-window_i][3]
        if(window_i!=1):pair_data.loc[0,"volume-"+str(window_i)+"_day"]=ohlcv1d[-window_i][4]  

    for window_i in range(1,window+1):
        pair_data.loc[0,"high-"+str(window_i)+"_hour"]=ohlcv1h[-window_i][1]
        pair_data.loc[0,"low-"+str(window_i)+"_hour"]=ohlcv1h[-window_i][2]
        pair_data.loc[0,"open-"+str(window_i)+"_hour"]=ohlcv1h[-window_i][0]
        pair_data.loc[0,"close-"+str(window_i)+"_hour"]=ohlcv1h[-window_i][3]
        if(window_i!=1):pair_data.loc[0,"volume-"+str(window_i)+"_hour"]=ohlcv1h[-window_i][4]  

    for window_i in range(1,window+1):
        pair_data.loc[0,"high-"+str(window_i)+"_15min"]=ohlcv15m[-window_i][1]
        pair_data.loc[0,"low-"+str(window_i)+"_15min"]=ohlcv15m[-window_i][2]
        pair_data.loc[0,"open-"+str(window_i)+"_15min"]=ohlcv15m[-window_i][0]
        pair_data.loc[0,"close-"+str(window_i)+"_15min"]=ohlcv15m[-window_i][3]
        if(window_i!=1):pair_data.loc[0,"volume-"+str(window_i)+"_15min"]=ohlcv5m[-window_i][4]  

    for window_i in range(1,window+1):
        pair_data.loc[0,"high-"+str(window_i)+"_5min"]=ohlcv5m[-window_i][1]
        pair_data.loc[0,"low-"+str(window_i)+"_5min"]=ohlcv5m[-window_i][2]
        pair_data.loc[0,"open-"+str(window_i)+"_5min"]=ohlcv5m[-window_i][0]
        pair_data.loc[0,"close-"+str(window_i)+"_5min"]=ohlcv5m[-window_i][3]
        if(window_i!=1):pair_data.loc[0,"volume-"+str(window_i)+"_5min"]=ohlcv5m[-window_i][4] 

    return  pair_data

def instant_full_data(pair,exchange=ex,window=WINDOW_SIZE):
    start = timeit.default_timer()

    pdata=instant_pair_data(pair,exchange=exchange,window=WINDOW_SIZE)
    btcdata=instant_pair_data("BTC/USDT",exchange=exchange,window=WINDOW_SIZE).add_prefix("BTC_")
    Timestamp=pd.to_datetime(ex.fetchTime(),unit='ms')
    pdata=pd.concat([pdata,btcdata],axis=1)
    pdata.loc[0,"day"]=Timestamp.dayofweek+1
    pdata.loc[0,"hour"]=Timestamp.hour
    pdata.loc[0,"minute"]=Timestamp.minute
    stop = timeit.default_timer()
    print("synctime for "+str(pair)+" :"+str(stop-start))
    try:
        pdata.loc[0,"lunch_day"]=int(-(pd.to_datetime(MetaData[MetaData["Pair"] == (pair.split("/")[0]+"/USDT")]["launch_minute"])-pd.Timestamp('2020-01-01 00:00:00.000000')).dt.days)
    except:
        MData=get_crypto_metadata([pair.split("/")[0]+"/USDT"])
        pdata.loc[0,"lunch_day"]=int(-(pd.to_datetime(MData[MData["Pair"] == (pair.split("/")[0]+"/USDT")]["launch_minute"])-pd.Timestamp('2020-01-01 00:00:00.000000')).dt.days)
    for key in pdata.keys():
        if key.find("BTC")!=-1 and (key.find("open")!=-1 or
        key.find("high")!=-1 or key.find("low")!=-1 or key.find("close")!=-1):
            pdata[key]=(pdata["BTC_price"]-pdata[key])/pdata["BTC_price"]
        if key.find("BTC")==-1 and (key.find("open")!=-1 or
        key.find("high")!=-1 or key.find("low")!=-1 or key.find("close")!=-1):
            pdata[key]=(pdata["price"]-pdata[key])/pdata["price"]
    return pdata


import asyncio
PRERR=False


import timeit

window=WINDOW_SIZE
async def do_ohlcv1m(pair,window,ohlcv1m):
    # prerr("Preparing 1 minute candelstics")
    await asyncio.sleep(0)
    ohlcv1m.extend(ex.fetch_ohlcv(pair, '1m', limit=window))
    # prerr("-->  1 minute candelstics is ok <--")
async def do_ohlcv5m(pair,window,ohlcv1m):
    # prerr("Preparing 5 minutes candelstics")
    await asyncio.sleep(0)
    ohlcv1m.extend(ex.fetch_ohlcv(pair, '5m', limit=window))
    # prerr("-->  5 minutes candelstics is ok <--")
async def do_ohlcv15m(pair,window,ohlcv1m):
    # prerr("Preparing 15 minutes candelstics")
    await asyncio.sleep(0)
    ohlcv1m.extend(ex.fetch_ohlcv(pair, '15m', limit=window))
    # prerr("-->  15 minutes candelstics is ok <--")
async def do_ohlcv1h(pair,window,ohlcv1m):
    # prerr("Preparing 1 hour candelstics")
    await asyncio.sleep(0)
    ohlcv1m.extend(ex.fetch_ohlcv(pair, '1h', limit=window))
    # prerr("-->  1 hour candelstics is ok <--")
async def do_ohlcv1d(pair,window,ohlcv1m):
    # prerr("Preparing 1 day candelstics")
    await asyncio.sleep(0)
    ohlcv1m.extend(ex.fetch_ohlcv(pair, '1d', limit=window))
    # prerr("-->  1 day candelstics is ok <--")

async def do_get_ask_price(pair,price):
    # prerr("Preparing 1 day candelstics")
    await asyncio.sleep(0)
    ticker = ex.fetch_ticker(pair)
    price[0]+=float(ticker['info']['askPrice'])
    # prerr("-->  1 day candelstics is ok <--")



async def async_instant_pair_data(pair="GMT/BUSD",exchange=ccxt.binance(),window=WINDOW_SIZE,pdata=pd.DataFrame()):
    #pair_data=pd.DataFrame()    
    
    pair_data=pdata
    ex=exchange

    ppp=[float(0)]
    #print(pair_current_price)
    ohlcv1m = []
    ohlcv5m = []
    ohlcv15m = []
    ohlcv1h = []
    ohlcv1d = []

    task_askprice = asyncio.create_task(do_get_ask_price(pair,ppp))
    task_ohlcv1m = asyncio.create_task(do_ohlcv1m(pair,window,ohlcv1m))
    task_ohlcv5m = asyncio.create_task(do_ohlcv5m(pair,window,ohlcv5m))
    task_ohlcv15m = asyncio.create_task(do_ohlcv15m(pair,window,ohlcv15m))
    task_ohlcv1h = asyncio.create_task(do_ohlcv1h(pair,window,ohlcv1h))
    task_ohlcv1d = asyncio.create_task(do_ohlcv1d(pair,window,ohlcv1d))

    await asyncio.wait([task_askprice,task_ohlcv1m,task_ohlcv5m,task_ohlcv15m,task_ohlcv1h,task_ohlcv1d])
    pair_current_price=ppp[0]
    #print("task1d   "+str(ohlcv1d))

    pair_data.loc[0,"price"]=float(pair_current_price)
    #minute
    for window_i in range(1,window+1):
        pair_data.loc[0,"high-"+str(window_i)]=ohlcv1m[-window_i][1]
        pair_data.loc[0,"low-"+str(window_i)]=ohlcv1m[-window_i][2]
        pair_data.loc[0,"open-"+str(window_i)]=ohlcv1m[-window_i][0]
        pair_data.loc[0,"close-"+str(window_i)]=ohlcv1m[-window_i][3]
        if(window_i!=1):pair_data.loc[0,"volume-"+str(window_i)]=ohlcv1m[-window_i][4]

    for window_i in range(1,window+1):
        pair_data.loc[0,"high-"+str(window_i)+"_day"]=ohlcv1d[-window_i][1]
        pair_data.loc[0,"low-"+str(window_i)+"_day"]=ohlcv1d[-window_i][2]
        pair_data.loc[0,"open-"+str(window_i)+"_day"]=ohlcv1d[-window_i][0]
        pair_data.loc[0,"close-"+str(window_i)+"_day"]=ohlcv1d[-window_i][3]
        if(window_i!=1):pair_data.loc[0,"volume-"+str(window_i)+"_day"]=ohlcv1d[-window_i][4]  

    for window_i in range(1,window+1):
        pair_data.loc[0,"high-"+str(window_i)+"_hour"]=ohlcv1h[-window_i][1]
        pair_data.loc[0,"low-"+str(window_i)+"_hour"]=ohlcv1h[-window_i][2]
        pair_data.loc[0,"open-"+str(window_i)+"_hour"]=ohlcv1h[-window_i][0]
        pair_data.loc[0,"close-"+str(window_i)+"_hour"]=ohlcv1h[-window_i][3]
        if(window_i!=1):pair_data.loc[0,"volume-"+str(window_i)+"_hour"]=ohlcv1h[-window_i][4]  

    for window_i in range(1,window+1):
        pair_data.loc[0,"high-"+str(window_i)+"_15min"]=ohlcv15m[-window_i][1]
        pair_data.loc[0,"low-"+str(window_i)+"_15min"]=ohlcv15m[-window_i][2]
        pair_data.loc[0,"open-"+str(window_i)+"_15min"]=ohlcv15m[-window_i][0]
        pair_data.loc[0,"close-"+str(window_i)+"_15min"]=ohlcv15m[-window_i][3]
        if(window_i!=1):pair_data.loc[0,"volume-"+str(window_i)+"_15min"]=ohlcv5m[-window_i][4]  

    for window_i in range(1,window+1):
        pair_data.loc[0,"high-"+str(window_i)+"_5min"]=ohlcv5m[-window_i][1]
        pair_data.loc[0,"low-"+str(window_i)+"_5min"]=ohlcv5m[-window_i][2]
        pair_data.loc[0,"open-"+str(window_i)+"_5min"]=ohlcv5m[-window_i][0]
        pair_data.loc[0,"close-"+str(window_i)+"_5min"]=ohlcv5m[-window_i][3]
        if(window_i!=1):pair_data.loc[0,"volume-"+str(window_i)+"_5min"]=ohlcv5m[-window_i][4]   
    return  pair_data



async def do_pdata(pair,exchange,window=WINDOW_SIZE,pdata=pd.DataFrame()):
    # prerr("Preparing 1 day candelstics")
    await async_instant_pair_data(pair=pair,exchange=exchange,window=window,pdata=pdata)
    # prerr("-->  1 day candelstics is ok <--")
async def do_whattime(TimestampAll):
    TimestampAll.append(pd.to_datetime(ex.fetchTime(),unit='ms'))
    
async def async_instant_full_data(pair,exchange=ex,window=WINDOW_SIZE):
    start = timeit.default_timer()
    pdata=pd.DataFrame()
    btcdata=pd.DataFrame()
    #async tascs
    TimestampAll=[]
    task_pdata=asyncio.create_task(do_pdata(pair=pair,exchange=exchange,window=window,pdata=pdata))
    task_btcdata=asyncio.create_task(do_pdata(pair="BTC/USDT",exchange=exchange,window=window,pdata=btcdata))
    task_whattime=asyncio.create_task(do_whattime(TimestampAll)) 
    #Timestamp=pd.to_datetime(ex.fetchTime(),unit='ms')
    await asyncio.wait([task_pdata, task_btcdata, task_whattime])
    Timestamp=TimestampAll[0]
    btcdata=btcdata.add_prefix("BTC_")
    pdata=pd.concat([pdata,btcdata],axis=1)
    pdata.loc[0,"day"]=Timestamp.dayofweek+1
    pdata.loc[0,"hour"]=Timestamp.hour
    pdata.loc[0,"minute"]=Timestamp.minute
    stop = timeit.default_timer()
    print("async time for "+str(pair)+" :"+str(stop-start))
    try:
        pdata.loc[0,"lunch_day"]=int(-(pd.to_datetime(MetaData[MetaData["Pair"] == (pair.split("/")[0]+"/USDT")]["launch_minute"])-pd.Timestamp('2020-01-01 00:00:00.000000')).dt.days)
    except:
        MData=get_crypto_metadata([pair.split("/")[0]+"/USDT"])
        pdata.loc[0,"lunch_day"]=int(-(pd.to_datetime(MData[MData["Pair"] == (pair.split("/")[0]+"/USDT")]["launch_minute"])-pd.Timestamp('2020-01-01 00:00:00.000000')).dt.days)
    for key in pdata.keys():
        if key.find("BTC")!=-1 and (key.find("open")!=-1 or
        key.find("high")!=-1 or key.find("low")!=-1 or key.find("close")!=-1):
            pdata[key]=(pdata["BTC_price"]-pdata[key])/pdata["BTC_price"]
        if key.find("BTC")==-1 and (key.find("open")!=-1 or
        key.find("high")!=-1 or key.find("low")!=-1 or key.find("close")!=-1):
            pdata[key]=(pdata["price"]-pdata[key])/pdata["price"]
    return pdata

async def do_async_instant_full_data(pdata,pair,exchange=ex,window=WINDOW_SIZE):
    
    start = timeit.default_timer()
    #pdata=pd.DataFrame()
    btcdata=pd.DataFrame()
    #async tascs
    TimestampAll=[]
    task_pdata=asyncio.create_task(do_pdata(pair=pair,exchange=exchange,window=window,pdata=pdata))
    task_btcdata=asyncio.create_task(do_pdata(pair="BTC/USDT",exchange=exchange,window=window,pdata=btcdata))
    task_whattime=asyncio.create_task(do_whattime(TimestampAll)) 
    #Timestamp=pd.to_datetime(ex.fetchTime(),unit='ms')
    await asyncio.wait([task_pdata, task_btcdata, task_whattime])
    Timestamp=TimestampAll[0]
    btcdata=btcdata.add_prefix("BTC_")
    #pdata=pd.concat([pdata,btcdata],axis=1,copy=False)
    #pdata.join(btcdata)
    for k in btcdata.keys():
        pdata.loc[0,str(k)]=btcdata[str(k)][0]
    pdata.loc[0,"day"]=Timestamp.dayofweek+1
    pdata.loc[0,"hour"]=Timestamp.hour
    pdata.loc[0,"minute"]=Timestamp.minute
    stop = timeit.default_timer()
    print("async time for "+str(pair)+" :"+str(stop-start))
    try:
        pdata.loc[0,"lunch_day"]=int(-(pd.to_datetime(MetaData[MetaData["Pair"] == (pair.split("/")[0]+"/USDT")]["launch_minute"])-pd.Timestamp('2020-01-01 00:00:00.000000')).dt.days)
    except:
        MData=get_crypto_metadata([pair.split("/")[0]+"/USDT"])
        pdata.loc[0,"lunch_day"]=int(-(pd.to_datetime(MData[MData["Pair"] == (pair.split("/")[0]+"/USDT")]["launch_minute"])-pd.Timestamp('2020-01-01 00:00:00.000000')).dt.days)
    for key in pdata.keys():
        if key.find("BTC")!=-1 and (key.find("open")!=-1 or
        key.find("high")!=-1 or key.find("low")!=-1 or key.find("close")!=-1):
            pdata[key]=(pdata["BTC_price"]-pdata[key])/pdata["BTC_price"]
        if key.find("BTC")==-1 and (key.find("open")!=-1 or
        key.find("high")!=-1 or key.find("low")!=-1 or key.find("close")!=-1):
            pdata[key]=(pdata["price"]-pdata[key])/pdata["price"]

async def get_all_data_for(list_pair=Binance_USDT_HALAL,exchange=ex,window=WINDOW_SIZE):
    #loop init data dic
    data_dic={}
    for pair in list_pair:
        data_dic[pair]=pd.DataFrame()
    
    ###loop tasklist
    task_list=[]
    for pair in list_pair:
        task_list.append(   asyncio.create_task(do_async_instant_full_data(pair=pair,exchange=exchange,window=window,pdata=data_dic[pair]))   )
    await asyncio.wait(task_list)

    return(data_dic)


In [32]:
#import ccxt.async_support as ccxt2

In [33]:
pdata=await async_instant_pair_data(pair="GMT/BUSD",exchange=ex,window=WINDOW_SIZE)
## minimum 1.4 seconds

In [34]:
pdata=instant_pair_data(pair="GMT/BUSD",exchange=ex,window=WINDOW_SIZE)
## minimum 1.6 seconds

In [35]:
pdata[~pdata.isnull().any(axis=1) ]

,price,high-1,low-1,open-1,close-1,high-2,low-2,open-2,close-2,volume-2,...,high-14_5min,low-14_5min,open-14_5min,close-14_5min,volume-14_5min,high-15_5min,low-15_5min,open-15_5min,close-15_5min,volume-15_5min
0,0.6828,0.6824,0.6829,1.664301e+12,0.6815,0.6833,0.6837,1.664300e+12,0.6823,0.6826,...,0.6998,0.6998,1.664296e+12,0.6949,0.6958,0.7015,0.7033,1.664296e+12,0.6982,0.7


In [62]:
pdata=await async_instant_full_data("MOB/USDT",exchange=ex,window=WINDOW_SIZE)

async time for MOB/USDT :5.350128131976817


In [49]:
pdata.shape


(1, 996)

In [38]:
# pdata=instant_full_data("MOB/USDT",exchange=ex,window=WINDOW_SIZE)

In [39]:
# pdata.shape

In [40]:
# data_dic=await get_all_data_for(list_pair=Binance_USDT_HALAL[1:5],exchange=ex,window=WINDOW_SIZE)

In [63]:
# pdata=data_dic['GMT/USDT']
# pdata=pdata[~pdata.isnull().any(axis=1)]
Buy_Dessision(pdata.iloc[0])

1/1 [==============================] - 0s 30ms/step


1

In [64]:

GoodDeal=[]
BadDeal=[]
pair="ETH/USDT"
min_win_percent=BUY_PERCENT
max_time_window=MAX_FORCAST_SIZE
window=WINDOW_SIZE
while True:
    break_out_flag = False
    for pair in Binance_USDT_HALAL[2:13]:
        pdata=await async_instant_full_data(pair,exchange=ex,window=WINDOW_SIZE)
        if int(Buy_Dessision(pdata.iloc[0])) == 1:
            print("Good : " +pair)
            print("buying at"+str(pdata["price"].iloc[0]))
            bt=pd.to_datetime(ex.fetchTime(),unit='ms')
            pp=0
            for i in range(max_time_window*2):
                time.sleep(30)
                pp=ex.fetch_ticker(pair)['info']['askPrice']
                if((float(pdata.loc[0,"price"])*0.01*min_win_percent+float(pdata.loc[0,"price"])) <= float(pp)):
                    GoodDeal.append({"pair":pair,
                                     "buying_time":bt,
                                     "buying_price":float(pdata["price"].iloc[0]),
                                     "Selling_time":pd.to_datetime(ex.fetchTime(),unit='ms'),
                                     "selling_price":float(pp)})
                    print("+++ wining bought at:"+str(pdata.loc[0,"price"]) +" sold at: "+str(pp) )
                    break_out_flag = True
                    break       
            if break_out_flag:break
            pp=ex.fetch_ticker(pair)['info']['askPrice']
            print("--- losing bought at: "+str(pdata.loc[0,"price"]) +" sold at: "+str(pp) )
            BadDeal.append({"pair":pair,
                                     "buying_time":bt,
                                     "buying_price":float(pdata["price"].iloc[0]),
                                     "Selling_time":pd.to_datetime(ex.fetchTime(),unit='ms'),
                                     "selling_price":float(pp)})
            
    #time.sleep(30)


async time for GMT/USDT :6.590874129033182
1/1 [==============================] - 0s 26ms/step
Good : GMT/USDT
buying at0.6877
--- losing bought at: 0.6877 sold at: 0.68800000
async time for UST/USDT :5.258025192015339
1/1 [==============================] - 0s 29ms/step
async time for SOL/USDT :3.9809603660251014
1/1 [==============================] - 0s 27ms/step
Good : SOL/USDT
buying at32.73
--- losing bought at: 32.73 sold at: 32.59000000
async time for APE/USDT :7.102871286973823
1/1 [==============================] - 0s 33ms/step
Good : APE/USDT
buying at5.369
--- losing bought at: 5.369 sold at: 5.36900000
async time for XRP/USDT :7.062607981031761
1/1 [==============================] - 0s 27ms/step
Good : XRP/USDT
buying at0.443
--- losing bought at: 0.443 sold at: 0.44480000
async time for IDEX/USDT :5.235677877033595
1/1 [==============================] - 0s 27ms/step
Good : IDEX/USDT
buying at0.06129


In [43]:
GoodDeal

[]